# Advent of Code

## 2015-012-022
## 2015 022

https://adventofcode.com/2015/day/22

In [4]:
import heapq

# Define spell costs and durations
SPELLS = {
    "Magic Missile": {"cost": 53, "damage": 4, "heal": 0, "armor": 0, "mana": 0, "duration": 0},
    "Drain": {"cost": 73, "damage": 2, "heal": 2, "armor": 0, "mana": 0, "duration": 0},
    "Shield": {"cost": 113, "damage": 0, "heal": 0, "armor": 7, "mana": 0, "duration": 6},
    "Poison": {"cost": 173, "damage": 3, "heal": 0, "armor": 0, "mana": 0, "duration": 6},
    "Recharge": {"cost": 229, "damage": 0, "heal": 0, "armor": 0, "mana": 101, "duration": 5},
}

# Define the Player class
class Player:
    def __init__(self, hp, mana):
        self.hp = hp
        self.mana = mana
        self.armor = 0

    def apply_effect(self, effect):
        if effect == "Shield":
            self.armor = SPELLS[effect]["armor"]
        elif effect == "Recharge":
            self.mana += SPELLS[effect]["mana"]

    def reset_armor(self):
        self.armor = 0

    def copy(self):
        return Player(self.hp, self.mana)

# Define the Boss class
class Boss:
    def __init__(self, hp, damage):
        self.hp = hp
        self.damage = damage

    def take_damage(self, damage):
        self.hp -= damage

    def copy(self):
        return Boss(self.hp, self.damage)

# Apply active effects
def apply_effects(player, boss, effects):
    player.reset_armor()
    for spell, timer in list(effects.items()):
        if timer > 0:
            if spell == "Poison":
                boss.take_damage(SPELLS[spell]["damage"])
            player.apply_effect(spell)
            effects[spell] -= 1

# Simulate the game
def simulate(player, boss):
    # Priority queue: (mana_spent, (player, boss, effects, is_player_turn))
    pq = [(0, (player, boss, {}, True))]
    min_mana = float("inf")

    while pq:
        mana_spent, (player, boss, effects, is_player_turn) = heapq.heappop(pq)

        # If mana spent exceeds current minimum, skip this path
        if mana_spent >= min_mana:
            continue

        # Apply effects
        apply_effects(player, boss, effects)

        # Check for win/loss
        if boss.hp <= 0:
            min_mana = min(min_mana, mana_spent)
            continue
        if player.hp <= 0:
            continue

        if is_player_turn:
            # Player's turn: try each spell
            for spell, data in SPELLS.items():
                if spell in effects and effects[spell] > 0:
                    continue  # Can't cast an active effect
                if player.mana < data["cost"]:
                    continue  # Can't afford this spell

                # Cast the spell
                new_player = player.copy()
                new_boss = boss.copy()
                new_effects = effects.copy()
                new_player.mana -= data["cost"]
                new_mana_spent = mana_spent + data["cost"]

                if data["duration"] > 0:
                    new_effects[spell] = data["duration"]
                else:
                    new_boss.take_damage(data["damage"])
                    new_player.hp += data["heal"]

                heapq.heappush(pq, (new_mana_spent, (new_player, new_boss, new_effects, False)))
        else:
            # Boss's turn: attack
            damage = max(1, boss.damage - player.armor)
            new_player = player.copy()
            new_player.hp -= damage
            heapq.heappush(pq, (mana_spent, (new_player, boss, effects, True)))

    return min_mana

# Parse boss stats from input.txt
def parse_boss_stats(filename):
    with open(filename, 'r') as file:
        lines = file.read().strip().splitlines()
        hp = int(lines[0].split(": ")[1])
        damage = int(lines[1].split(": ")[1])
    return Boss(hp, damage)

# Main logic
filename = "input.txt"
boss = parse_boss_stats(filename)
player = Player(50, 500)

# Find the least mana spent to win
min_mana_spent = simulate(player, boss)
print(f"The least amount of mana spent to win is: {min_mana_spent}")


TypeError: '<' not supported between instances of 'Player' and 'Player'